Copyright (c) 2023 Habana Labs, Ltd. an Intel Company.

#### Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Using Paramater Efficient Fine Tuning on Llama2
This example will Fine Tune the Llama2-7B model using Parameter Efficient Fine Tuining (PEFT) and then run inference on a text prompt.  This will be using the Llama2 model with two task examples from the Optimum Habana library on the Hugging Face model repository.   The Optimum Habana library is optimized for Deep Learning training and inference on First-gen Gaudi and Gaudi2 and offers tasks such as text generation, language modeling, question answering and more. For all the examples and models, please refer to the [Optimum Habana GitHub](https://github.com/huggingface/optimum-habana#validated-models).

This example will Fine Tune the Llama2-7B model using Parameter Efficient Fine Tuining (PEFT) on the timdettmers/openassistant-guanaco dataset using the Language-Modeling Task in Optimum Habana.

### Parameter Efficient Fine Tuning with Low Rank Adaptatoin
Parameter Efficient Fine Tuning is a strategy for adapting large pre-trained language models to specific tasks while minimizing computational and memory demands.   It aims to reduce the computational cost and memory requirements associated with fine-tuning large models while maintaining or even improving their performance.  It does so by adding a smaller task-specific layer, leveraging knowledge distillation, and often relying on few-shot learning, resulting in efficient yet effective models for various natural language understanding tasks.   PEFT starts with a pre-trained language model that has already learned a wide range of language understanding tasks from a large corpus of text data. These models are usually large and computationally expensive.   Instead of fine-tuning the entire pre-trained model, PEFT adds a task-specific layer or a few task-specific layers on top of the pre-trained model. These additional layers are relatively smaller and have fewer parameters compared to the base model.


In [1]:
%cd /root/Gaudi-tutorials/PyTorch/llama2_fine_tuning_inference

/root/Gaudi-tutorials/PyTorch/llama2_fine_tuning_inference


### Model Setup: 

##### Install the latest versoin of the Habana Deepspeed Library

In [2]:
!pip install -q git+https://github.com/HabanaAI/DeepSpeed.git

##### Install the Parameter Efficient Fine Tuning Library methods
This is taking the PEFT method from the Hugging Face repository and will be used to help create the PEFT Fine Tuning with the Llama2 model.

In [3]:
!git clone https://github.com/huggingface/peft.git
%cd peft
!pip install -q .
%cd ..

Cloning into 'peft'...
remote: Enumerating objects: 4772, done.
remote: Counting objects: 100% (1687/1687), done.
remote: Compressing objects: 100% (450/450), done.
remote: Total 4772 (delta 1412), reused 1342 (delta 1192), pack-reused 3085
Receiving objects: 100% (4772/4772), 8.60 MiB | 30.67 MiB/s, done.
Resolving deltas: 100% (3107/3107), done.
/root/Gaudi-tutorials/PyTorch/llama2_fine_tuning_inference/peft
/root/Gaudi-tutorials/PyTorch/llama2_fine_tuning_inference


##### Install the Optimum-Habana Library

In [4]:
!pip install -q --upgrade-strategy eager optimum[habana]

##### Pull the Hugging Face Examples from GitHub
These contain the working Hugging Face Task Examples that have been optimized for Gaudi.  For Fine Tuning, we'll use the language-modeling task. 

In [5]:
!git clone https://github.com/huggingface/optimum-habana

Cloning into 'optimum-habana'...
remote: Enumerating objects: 5995, done.
remote: Counting objects: 100% (3034/3034), done.
remote: Compressing objects: 100% (1071/1071), done.
remote: Total 5995 (delta 2415), reused 2200 (delta 1919), pack-reused 2961
Receiving objects: 100% (5995/5995), 3.03 MiB | 16.33 MiB/s, done.
Resolving deltas: 100% (3856/3856), done.


##### Go to the Language Modeling Task and install the model specific requirements

In [6]:
%cd optimum-habana/examples/language-modeling/
!pip install -q -r requirements.txt

/root/Gaudi-tutorials/PyTorch/llama2_fine_tuning_inference/optimum-habana/examples/language-modeling


##### How to access and Use the Llama2 model

Use of the pretrained model is subject to compliance with third party licenses, including the “Llama 2 Community License Agreement” (LLAMAV2). For guidance on the intended use of the LLAMA2 model, what will be considered misuse and out-of-scope uses, who are the intended users and additional terms please review and read the instructions in this link https://ai.meta.com/llama/license/.
Users bear sole liability and responsibility to follow and comply with any third party licenses, and Habana Labs disclaims and will bear no liability with respect to users’ use or compliance with third party licenses.

To be able to run gated models like this Llama-2-7b-hf, you need the following: 
- Have a HuggingFace account
- Agree to the terms of use of the model in its model card on the HF Hub
- set a read token
- Login to your account using the HF CLI: run huggingface-cli login before launching your script

In [7]:
!huggingface-cli login --token <your token here>

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Fine Tuning the model with PEFT and LoRA

We'll now run the fine tuning with the PEFT method. Remember that the PEFT methods only fine-tune a small number of extra model parameters, thereby greatly decreasing the computational and storage costs. Recent State-of-the-Art PEFT techniques achieve performance comparable to that of full fine-tuning.

##### Here's a summary of the command required to run the Fine Tuning, you'll run this in the next cell below. 
Note in this case the following: 
1. Using the language modeling with LoRA; `run_lora_clm.py`
2. It's very efficient: only 0.06% of the total paramters are being fine tuned of the total 7B parameters.
3. Using DeepSpeed has reduced the max amount of memory to 31.03 GB out of a total memory available 94.61 GB
4. Only 2 epochs are needed for fine tuning, it takes less than 6 minutes to run. 

```
python ../gaudi_spawn.py --use_deepspeed \
       --world_size 8 run_lora_clm.py \
       --model_name_or_path meta-llama/Llama-2-7b-hf  \
       --dataset_name timdettmers/openassistant-guanaco \
       --bf16 True \
       --output_dir ./model_lora_llama \
       --num_train_epochs 2 \
       --per_device_train_batch_size 2 \
       --per_device_eval_batch_size 2 \
       --gradient_accumulation_steps 4 \
       --evaluation_strategy "no"\
       --save_strategy "steps"\
       --save_steps 2000 \
       --save_total_limit 1 \
       --learning_rate 1e-4 \
       --logging_steps 1 \
       --dataset_concatenation \
       --do_train \
       --use_habana \
       --use_lazy_mode \
       --throughput_warmup_steps 3
```

In [8]:
!python ../gaudi_spawn.py --use_deepspeed --world_size 8 run_lora_clm.py --model_name_or_path meta-llama/Llama-2-7b-hf  --dataset_name timdettmers/openassistant-guanaco --bf16 True --output_dir ./model_lora_llama --num_train_epochs 2 --per_device_train_batch_size 2 --per_device_eval_batch_size 2 --gradient_accumulation_steps 4 --evaluation_strategy "no" --save_strategy "steps" --save_steps 2000 --save_total_limit 1 --learning_rate 1e-4 --logging_steps 1 --dataset_concatenation --do_train --use_habana --use_lazy_mode --throughput_warmup_steps 3

/usr/local/lib/python3.8/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
/usr/local/lib/python3.8/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
[2023-10-10 03:19:00,239] [INFO] [real_accelerator.py:123:get_accelerator] Setting ds_accelerator to hpu (auto detect)
[WARNING|utils.py:177] 2023-10-10 03:19:01,352 >> optimum-habana v1.7.5 has been validated for SynapseAI v1.11.0 but habana-frameworks v1.12.0.480 was found, this could lead to undefined behavi

#### LoRA Fine Tuning Completed
You will now see a "model_lora_llama" folder created which contains the PEFT model `adapter_model.bin` which will be used in the inference example below. 

## Inference with Llama2

We'll now use the Hugging Face `text-generation` task to run inference on the Llama2-7b model; we'll generate text based on an included prompt.  Notice that we've included a path to the PEFT model that we just created.

First, well move to the text-generation examples folder and install the requirements. 

In [9]:
%cd /root/Gaudi-tutorials/PyTorch/llama2_fine_tuning_inference/optimum-habana/examples/text-generation
!pip install -q -r requirements.txt

/root/Gaudi-tutorials/PyTorch/llama2_fine_tuning_inference/optimum-habana/examples/text-generation


You will see that we are now running inference with the `run_generation.py` task and we are including the PEFT model that we Fine Tuned in the steps above. 

```
python run_generation.py \
   --model_name_or_path meta-llama/Llama-2-7b-hf \
   --batch_size 1 \
   --do_sample
   --max_new_tokens 500 \
   --n_iterations 4 \
   --use_kv_cache \
   --use_hpu_graphs \
   --bf16 \
   --prompt "I am a dog. Please help me plan a surprise birthday party for my human, including fun activities, games and decorations. And don't forget to order a big bone-shaped cake for me to share with my fur friends!" \
   --peft_model /root/Gaudi-tutorials/PyTorch/llama2_fine_tuning_inference/optimum-habana/examples/language-modeling/model_lora_llama/
```

In [28]:
prompt = input("Enter a prompt for text generation: ")

Enter a prompt for text generation:  I am a dog. Please help me plan a surprise birthday party for my human, including fun activities, games and decorations. And don't forget to order a big bone-shaped cake for me to share with my fur friends!


In [29]:
cmd = f'python run_generation.py --model_name_or_path meta-llama/Llama-2-7b-hf --batch_size 1 --do_sample --max_new_tokens 250 --n_iterations 4 --use_kv_cache --use_hpu_graphs \
--bf16 --prompt "{prompt}" --peft_model /root/Gaudi-tutorials/PyTorch/llama2_fine_tuning_inference/optimum-habana/examples/language-modeling/model_lora_llama/'
print(cmd)
import os
os.system(cmd)

python run_generation.py --model_name_or_path meta-llama/Llama-2-7b-hf --batch_size 1 --do_sample --max_new_tokens 250 --n_iterations 4 --use_kv_cache --use_hpu_graphs --bf16 --prompt "I am a dog. Please help me plan a surprise birthday party for my human, including fun activities, games and decorations. And don't forget to order a big bone-shaped cake for me to share with my fur friends!" --peft_model /root/Gaudi-tutorials/PyTorch/llama2_fine_tuning_inference/optimum-habana/examples/language-modeling/model_lora_llama/


10/10/2023 05:22:41 - INFO - __main__ - Single-device run.
10/10/2023 05:22:42 - INFO - numexpr.utils - Note: detected 160 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
10/10/2023 05:22:42 - INFO - numexpr.utils - Note: NumExpr detected 160 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
10/10/2023 05:22:42 - INFO - numexpr.utils - NumExpr defaulting to 8 threads.


/usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
[2023-10-10 05:22:42,765] [INFO] [real_accelerator.py:123:get_accelerator] Setting ds_accelerator to hpu (auto detect)


/usr/local/lib/python3.8/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/usr/local/lib/python3.8/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
[WARNING|utils.py:177] 2023-10-10 05:22:43,849 >> optimum-habana v1.7.5 has been validated for SynapseAI v1.11.0 but habana-frameworks v1.12.0.480 was found, this could lead to undefined behavior!
[WARNING|utils.py:190] 2023-10-10 05:22:44,994 >> optimum-habana v1.7.5 has been validated for SynapseAI v1.11.0 but the driver version is v1.12.0, this could lead to undefined behavior!
Loading checkpoint shards: 100%|██████████|


Input/outputs:
input 1: ("I am a dog. Please help me plan a surprise birthday party for my human, including fun activities, games and decorations. And don't forget to order a big bone-shaped cake for me to share with my fur friends!",)
output 1: ("I am a dog. Please help me plan a surprise birthday party for my human, including fun activities, games and decorations. And don't forget to order a big bone-shaped cake for me to share with my fur friends!\nI'm sorry, but I'm not a dog, and I don't know how to plan a surprise birthday party. But I can give you some ideas for fun activities and games that your human might enjoy.\nHere are some fun activities and games that your human might enjoy:\nPuzzle games: Your human might enjoy playing puzzle games like jigsaw puzzles or logic puzzles. You could also look for games that involve building something, like a model airplane or a LEGO set.\nBoard games: Your human might enjoy playing board games like Monopoly, Clue, or Trivial Pursuit. You c

0

##### Comparison without PEFT and LoRA
In this example, we're simply running the Llama2 7B model **without** including the PEFT fine tuned model, so the you are losing the additional detail that is brought to the model, and the results have signficantly less information and fidelity compared to the last model.

In [30]:
cmd = f'python run_generation.py --model_name_or_path meta-llama/Llama-2-7b-hf --batch_size 1 --do_sample --max_new_tokens 250 --n_iterations 4 --use_kv_cache --use_hpu_graphs \
--bf16 --prompt "{prompt}"'
print(cmd)
import os
os.system(cmd)

python run_generation.py --model_name_or_path meta-llama/Llama-2-7b-hf --batch_size 1 --do_sample --max_new_tokens 250 --n_iterations 4 --use_kv_cache --use_hpu_graphs --bf16 --prompt "I am a dog. Please help me plan a surprise birthday party for my human, including fun activities, games and decorations. And don't forget to order a big bone-shaped cake for me to share with my fur friends!"


10/10/2023 05:24:11 - INFO - __main__ - Single-device run.
10/10/2023 05:24:12 - INFO - numexpr.utils - Note: detected 160 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
10/10/2023 05:24:12 - INFO - numexpr.utils - Note: NumExpr detected 160 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
10/10/2023 05:24:12 - INFO - numexpr.utils - NumExpr defaulting to 8 threads.


/usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
[2023-10-10 05:24:12,659] [INFO] [real_accelerator.py:123:get_accelerator] Setting ds_accelerator to hpu (auto detect)


/usr/local/lib/python3.8/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/usr/local/lib/python3.8/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
[WARNING|utils.py:177] 2023-10-10 05:24:13,742 >> optimum-habana v1.7.5 has been validated for SynapseAI v1.11.0 but habana-frameworks v1.12.0.480 was found, this could lead to undefined behavior!
[WARNING|utils.py:190] 2023-10-10 05:24:14,897 >> optimum-habana v1.7.5 has been validated for SynapseAI v1.11.0 but the driver version is v1.12.0, this could lead to undefined behavior!
Loading checkpoint shards: 100%|██████████|


Input/outputs:
input 1: ("I am a dog. Please help me plan a surprise birthday party for my human, including fun activities, games and decorations. And don't forget to order a big bone-shaped cake for me to share with my fur friends!",)
output 1: ("I am a dog. Please help me plan a surprise birthday party for my human, including fun activities, games and decorations. And don't forget to order a big bone-shaped cake for me to share with my fur friends!\nI am a cat. Please help me plan a surprise birthday party for my human, including fun activities, games and decorations. And don't forget to order a big cake for me to share with my fur friends! In this video I'll show you how to install and setup your new Dell laptop.\nIn this video I'll show you how to install and setup your new Dell laptop. This is a step-by-step video that will walk you through the installation process. A few weeks ago, I had a chance to take a quick trip to San Diego. I spent a few days in the city and then a few da

0

In [ ]:
exit()